In [135]:
# Import Libraries

import datetime
import sys
np.set_printoptions(threshold=sys.maxsize)
import numpy as np
import pandas as pd
import talib as ta    # https://github.com/mrjbq7/ta-lib
import pyswarms as ps
import matplotlib.pyplot as plt

# Read Initial Data

In [136]:
# Reading Data
class ohlcv_df:
    time = []
    open = []
    high = []
    low = []
    close = []
    volume = []

    def __init__(self, file_name):
        # Read and process the dataframe from file
        df = pd.read_feather(file_name)
        df.dropna(inplace=True)     # Dropping NA values, so that talib works correctly
        df.reset_index(drop=True, inplace=True)
        # Convert Time
        df['time'] = df['time']*1000000
        self.time = pd.to_datetime(df['time'])
        # Save arrays with data
        self.open = df['open'].to_numpy()
        self.high = df['high'].to_numpy()
        self.low = df['low'].to_numpy()
        self.close = df['close'].to_numpy()
        self.volume = df['volume'].to_numpy()

        pass

    # Get start and end ids for the data range
    def get_data_id(self, start_date, end_date):
        # Get Start id
        start_id = -1
        for i in range(len(self.time)):
            if self.time[i] == start_date:
                start_id = i
                break
        # Get End id
        end_id = -1
        for i in range(len(self.time)-1, 0, -1):
            if self.time[i] == end_date:
                end_id = i
                break

        return start_id, end_id
        
    # Get OHLCV data in given range
    def get_data(self, start_date, end_date):
        # Set start and end ids
        start_id, end_id = self.get_data_id(start_date, end_date)
        # Get data
        open = self.open[start_id: end_id]
        high = self.high[start_id: end_id]
        low = self.low[start_id: end_id]
        close = self.close[start_id: end_id]
        volume = self.volume[start_id: end_id]
        
        return open, high, low, close, volume


# Creating object with dataframe
df_bnb_usdt_1h = ohlcv_df("data-BNBUSDT1h.feather")

# Start/End data points in yyyy/mm/dd
d1 = datetime.datetime(2019, 1, 1)
d2 = datetime.datetime(2021, 1, 1)

# Setting OHLCV to work on
open, high, low, close, volume = df_bnb_usdt_1h.get_data(d1, d2)
size = len(volume)


# Calculate Stats

In [137]:
# Finds DD for long and short trades
# ToDo: Combine Longs and Shorts (Or just assign weight to the score formula based on the amount of trades)
def calculate_DD(trades_long, trades_short):

    # DD Long
    cumulitive_return_long = 1.0
    current_high = 1.0
    current_dd = 0.0
    current_dd_lowest = 0.0
    dd_long = []
    for trade in trades_long:
        cumulitive_return_long *= trade
        if current_high - cumulitive_return_long > 0.0:             # Portfolio goes lower
            current_dd = (current_high - cumulitive_return_long) / current_high
            if current_dd > current_dd_lowest:                      # Porfolio found new lowest point
                current_dd_lowest = current_dd
        if cumulitive_return_long > current_high and current_dd_lowest != 0.0:     # Portfolio goes up after DD
            dd_long.append(current_dd_lowest)
            current_dd = 0.0
            current_dd_lowest = 0.0
        if cumulitive_return_long > current_high:     # Portfolio goes up
            current_high = cumulitive_return_long
    if current_dd_lowest != 0.0: dd_long.append(current_dd_lowest)

    # DD Short
    cumulitive_return_short = 1.0
    current_high = 1.0
    current_dd = 0.0
    current_dd_lowest = 0.0
    dd_short = []
    for trade in trades_short:
        cumulitive_return_short *= trade
        if current_high - cumulitive_return_short > 0.0:             # Portfolio goes lower
            current_dd = (current_high - cumulitive_return_short) / current_high
            if current_dd > current_dd_lowest:                      # Porfolio found new lowest point
                current_dd_lowest = current_dd
        if cumulitive_return_short > current_high and current_dd_lowest != 0.0:     # Portfolio goes up after DD
            dd_short.append(current_dd_lowest)
            current_dd = 0.0
            current_dd_lowest = 0.0
        if cumulitive_return_short > current_high:     # Portfolio goes up
            current_high = cumulitive_return_short
    if current_dd_lowest != 0.0: dd_short.append(current_dd_lowest)

    return dd_long, dd_short


# Return/DD
# ToDo: Add weight to the formula, based on the amount of trades
def calculate_score(trades_long, trades_short, show_money = False):

    dd_long, dd_short = calculate_DD(trades_long, trades_short)

    # Finding Average Trade Size
    if len(trades_long) + len(trades_short) > 20: # If enough trades happened
        average_long_trades_res = np.sum(trades_long)/len(trades_long)
        average_short_trades_res = np.sum(trades_short)/len(trades_short)
        if show_money:
            #print("Long Trades: ", trades_long)
            print("Average Long Trade: ", average_long_trades_res)
            print("Amount of Long Trades: ", len(trades_long))
            print("DD Long: ", dd_long)
            print()            
            #print("Short Trades: ", trades_short)
            print("Average Short Trade: ", average_short_trades_res)
            print("Amount of Short Trades: ", len(trades_short))
            print("DD Short: ", dd_short)
        # (Longs/DD_long + Shorts/DD_short) / 2
        score = (average_long_trades_res/np.max(dd_long) + average_short_trades_res/np.max(dd_short)) / 2    # Gives same weight to long/short not overall trades
        return(-score)
    else: return(1) # If result incorrect, result is just positive as it is worse than any possible valid result


# Strategy

In [138]:

# Strategy Function
# Input Params is a List with 3 values:
#       ma_direction_period, atr_direction_period, atr_direction_coeff
# ToDo: Create Combine Trades List (Or should I?)
def strategy_ma(input_params, show_money = False):

    # Generate Indicators
    ma_direction_period = input_params[0]
    atr_direction_period = input_params[1]
    atr_direction_coeff = input_params[2]
    ma_direction = ta.EMA(close, timeperiod = ma_direction_period)
    atr_direction = ta.ATR(high, low, close, timeperiod = atr_direction_period)

    # Calculate Trades
    in_trade = 0    # +1 = long_entry, -1 = shor_entry
    entry_price = -1.0
    trades_long = []
    trades_short = []
    # Parse dataframe
    for i in range(size):
        # Long Entry
        if (ma_direction[i] - close[i] > atr_direction_coeff*atr_direction[i]
            and in_trade == 0.0):
            in_trade = +1.0
            entry_price = close[i]
        # Long Exit
        if (ma_direction[i] - close[i] < 0.0
            and in_trade == 1.0):
            in_trade = 0.0
            trades_long.append(close[i]/entry_price)

        # Short Entry
        if (ma_direction[i] - close[i] < -atr_direction_coeff*atr_direction[i]
            and in_trade == 0.0):
            in_trade = -1.0
            entry_price = close[i]
        # Short Exit
        if (ma_direction[i] - close[i] > 0.0
            and in_trade == -1.0):
            in_trade = 0.0
            trades_short.append(entry_price/close[i])

    result = calculate_score(trades_long, trades_short, show_money)

    return result


# Optimizer

In [139]:

# Optimizer Function
# Input Params are taylored for the strategy which we are optimizing
def optimize_swarms(input_params):

    results = []

    # Calculate results for each particle
    for i in range(len(input_params)):
    
        temp_res = strategy_ma(input_params[i])
        results.append(temp_res)

    return results

# Starting the Optimization

# Declaring Input for the Strategy
input_max = [192, 96, 6.0]
input_min = [8, 8, -2.0]
input_bounds = (input_min, input_max)

# Instatiate the optimizer
options = {'c1': 0.6, 'c2': 0.6, 'w': 0.8}
optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=3, options=options, bounds=input_bounds)
cost, pos = optimizer.optimize(optimize_swarms, 40)
#print(optimizer.mean_pbest_history)

# Show results with the best found params
strategy_ma([18.77582177, 83.92109873,  5.80824639], True)

2022-12-27 16:35:23,814 - pyswarms.single.global_best - INFO - Optimize for 40 iters with {'c1': 0.6, 'c2': 0.6, 'w': 0.8}
pyswarms.single.global_best: 100%|██████████|40/40, best_cost=-12.4
2022-12-27 16:35:59,139 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -12.365039266101356, best pos: [21.81879518 45.95620836  5.59124728]


Average Long Trade:  1.003818185469477
Amount of Long Trades:  11
DD Long:  [0.02274226804123697, 0.051782441619863964, 0.11331768313061852]

Average Short Trade:  1.0192916919445274
Amount of Short Trades:  12
DD Short:  [0.046506169011154216]


-15.387893139786055